In [8]:
import pandas as pd

codons = pd.read_csv('../data/CodonUsage_ecoli.csv')
codons = codons.set_index('codon')
print(codons.head(5))
lookup = codons.to_dict(orient='index')

      aa  f_aa   f_tot   F_fam      r_aa  hydrophob_7  aromatic     class
codon                                                                    
TAA    *  0.61  0.0020  0.0033  0.333333         0.00         0          
TAG    *  0.09  0.0003  0.0033  0.333333         0.00         0          
TGA    *  0.30  0.0010  0.0033  0.333333         0.00         0          
GCT    A  0.18  0.0171  0.0926  0.250000         0.41         0  nonpolar
GCC    A  0.26  0.0242  0.0926  0.250000         0.41         0  nonpolar


In [1]:
with open('../data/tmp_seq_rpoB-20kb.txt', 'r') as f:
    seq = f.read()

In [2]:
import re
from collections import defaultdict as ddict

class DNAFeatureExtraction:
    '''parent class for dna translation and feature initialization'''
    def __init__(self, seq):
        self.seq = seq.upper()
        self.frames = [0,1,2] # frames
        self.stops_reg = re.compile('|'.join(['TAA','TAG','TGA'])) # define stops  
        
    def stop_delimited_frames(self):
        '''Find/Split all possible stop-delimited coding frames\naggregate to single list'''
        elements = ddict(list) # initialize result container
        indicies = ddict(list) # initialize indicies container
        idx = {0:0,1:1,2:2} # starting frame indicies

        # progressively search for stop codons, increment result vectors by frame
        i = max(idx.values())
        while i < len(self.seq):
            x = re.search(self.stops_reg, self.seq[i:]) # find next stop
            if x:
                frame = (x.start()+i)%3 # establish frame
                sequence = self.seq[idx[frame]:x.end()+i] # grab sequence from previous frame end
        #         print(frame, x.group(), sequence)
                elements[frame].append(sequence)
                indicies[frame].append(idx[frame])
                idx[frame] = x.end()+i # update frame start index
                i = max(idx.values())-2 # set new search start (next frame)

            else: # cap all vectors with the remaining non-stopped sequence
                for x in self.frames:
                    sequence = self.seq[idx[x]:]
                    elements[x].append(sequence[:len(sequence)-(len(sequence)%3)]) # trim to full codons
                    indicies[x].append(idx[x])
                break
        # aggregate to single list
        self.fragments = [x for frame in elements.values() for x in frame]
        self.frag_index = [x for frame in indicies.values() for x in frame] # global index

    def decode_dna(self, lookup):
        '''translate dna sequence, compile derived AA features'''
        self.peptides = []
        self.peptide_id = []
        self.codon_pref = []
        self.aa_hydrophob = []
        self.aa_aromatic = []
        self.aa_class = []
        for j,seq in enumerate(self.fragments):
            prot = ''
            pid, pref, aa_h, aa_a, aa_c = [],[],[],[],[]
            for i in range(0,len(seq),3):
                entry = lookup[seq[i:i+3]]
                prot += entry['aa']
                # add'nl properties:
                pid.append(self.frag_index[j])
                pref.append(entry['f_aa']/entry['r_aa'])
                aa_h.append(entry['hydrophob_7'])
                aa_a.append(entry['aromatic'])
                aa_c.append(entry['class'])
            self.peptides.append(prot)
            self.peptide_id.append(pid)
            self.codon_pref.append(pref)
            self.aa_hydrophob.append(aa_h)
            self.aa_aromatic.append(aa_a)
            self.aa_class.append(aa_c)
    
    def zip_all(self):
        '''list of (amino_acid, codon_pref, hydrophob, aromatic, class, global_peptide_index)'''
        return [list(zip(aa,pref,aah,aaa,aac,pix)) 
                for aa,pref,aah,aaa,aac,pix 
                in zip(self.peptides,self.codon_pref,self.aa_hydrophob,
                       self.aa_aromatic,self.aa_class,self.peptide_id)]

In [6]:
seq_object = DNAFeatureExtraction(seq)
seq_object.stop_delimited_frames()
print(seq_object.fragments[:10])
print(seq_object.frag_index[:10])

['CATTACTGTCACAATTTCCAACAGCACCAAATTACCCCCAGGCGGATGGTTCAGTAA', 'AACTGGCAGCAGGTTGGCTTATCGATCAGTGCCAGCTAA', 'AAGGGATGCAAATAG', 'GTGGGGCTGCGGTGCACCGTCAACAGGCGTTAG', 'TTCTCATTAATGAAGACAATGCAAAAAGCGAAGATGTTGTACAGCTGGCGCATCATGTAA', 'GACAGAAAGTTGGTGAAAAATTTAATGTCTGGCTTGAGCCTGAAGTCCGCTTTATTGGTGCATCAGGTGAAGTGA', 'GCGCAGTGGAGACAATTTCATGAAGGATAA', 'CACCGTGCCACTGAAATTGATTGCCCTGTTAGCGAACGGTGA', 'ATTTCACTCTGGCGAGCAGTTGGGTGA', 'AACGCTGGGAATGAGCCGGGCGGCTATTAA']
[0, 57, 96, 111, 144, 204, 279, 309, 351, 378]


In [9]:
seq_object.decode_dna(lookup)
print(seq_object.peptides[:10])
print(seq_object.codon_pref[:4])
print(seq_object.zip_all()[:4])

['HYCHNFQQHQITPRRMVQ*', 'NWQQVGLSISAS*', 'KGCK*', 'VGLRCTVNRR*', 'FSLMKTMQKAKMLYSWRIM*', 'DRKLVKNLMSGLSLKSALLVHQVK*', 'AQWRQFHEG*', 'HRATEIDCPVSER*', 'ISLWRAVG*', 'NAGNEPGGY*']
[[1.14, 0.82, 0.92, 0.86, 0.98, 0.84, 0.68, 1.32, 0.86, 0.68, 1.4700000014699999, 1.6, 0.52, 0.23999999952, 0.65999999868, 1.0, 1.12, 1.32, 1.83000000183], [1.02, 1.0, 1.32, 1.32, 1.12, 1.48, 0.8399999983200002, 0.8399999983200002, 1.17000000117, 0.95999999808, 1.04, 1.4999999970000002, 1.83000000183], [0.52, 0.52, 1.08, 1.48, 0.27000000027], [1.4, 0.6, 2.81999999436, 0.65999999868, 1.08, 1.6, 0.8, 1.02, 0.23999999952, 2.1599999956800002, 0.27000000027]]
[[('H', 1.14, 0.08, 1, 'basic', 0), ('Y', 0.82, 0.63, 1, 'polar', 0), ('C', 0.92, 0.49, 0, 'polar', 0), ('H', 0.86, 0.08, 1, 'basic', 0), ('N', 0.98, -0.28, 0, 'polar', 0), ('F', 0.84, 1.0, 1, 'nonpolar', 0), ('Q', 0.68, -0.1, 0, 'polar', 0), ('Q', 1.32, -0.1, 0, 'polar', 0), ('H', 0.86, 0.08, 1, 'basic', 0), ('Q', 0.68, -0.1, 0, 'polar', 0), ('I', 1.47000000146